<h2>Crawling data</h2>

<p>code dibawah digunakan untuk mengcrawling data dari sebuah website yang nantinya akan diolah dengan stemming, tf-idf dan lian-lain</p>

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        start_urls = [
            'https://pta.trunojoyo.ac.id/welcome/index/10',
            'https://pta.trunojoyo.ac.id/welcome/index/11',
            'https://pta.trunojoyo.ac.id/welcome/index/12',
            'https://pta.trunojoyo.ac.id/welcome/index/13',
            'https://pta.trunojoyo.ac.id/welcome/index/14',
        ]

        for url in start_urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for item in response.css('#content_journal > ul > li'):
            yield {
                'Link': item.css(f'div:nth-child(3) > div:nth-child(5) > a::attr(href)').get(),
            }

<p>code diatas digunakan untuk mengambil link setiap dari website yang nantinya akan diproses lebih lebih lanjut menggunakan kode dibawah, data yang dihasilkan nantinya akan dikonversi ke dalam file csv<p>

In [ ]:
import scrapy
import pandas as pd


class QuotesSpider(scrapy.Spider):
    name = "quote"

    def start_requests(self):
        data_csv = pd.read_csv('hasil_crawl.csv').values
        start_urls = [ link[0] for link in data_csv ]

        for url in start_urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        yield {
            'Judul': response.css('#content_journal > ul > li > div:nth-child(2) > a::text').get(),
            'Abstraksi': response.css('#content_journal > ul > li > div:nth-child(4) > div:nth-child(2) > p::text').get(),
            
        }

<h1>Topic Modeling menggunakan LDA dan LSA dengan Jupyter Notebook</h1>

<p> <em>Latent Dirichlet Allocation</em> (LDA) adalah jenis pemodelan statistik untuk menemukan "topik" abstrak yang terjadi dalam kumpulan dokumen. Latent Dirichlet Allocation (LDA) adalah contoh model topik dan digunakan untuk mengklasifikasikan teks dalam dokumen ke topik tertentu. Itu membangun topik per model dokumen dan kata-kata per model topik, dimodelkan sebagai distribusi Dirichlet.</p>

<p> <em>Latent Semantic Analysis</em> (LSA) adalah salah satu teknik dasar dalam pemodelan topik. Ide intinya adalah mengambil matriks dari apa yang kita miliki — dokumen dan istilah — dan menguraikannya menjadi matriks topik-dokumen yang terpisah dan matriks topik-istilah.</p>

<h3>Install Library</h3>

<p>Dibawah ini dicantumkan beberapa library yang dibutuhkan untuk menlakukan processing menggunakan LSA dengan jupyter notebook, anda juga bisa melakukan instalasi ini dengan menggunakan CMD untuk pengguna Windows atau Terminal untuk pengguna Linux</p>

In [ ]:
pip install numpy
pip install sklearn
pip install pandas
pip install matplotlib
pip install seaborn
pip install nltk

<h3>Import Library</h3>

<p>Untuk library yang digunakan diantaranya ada numpy, pandas, matplotlib, seaborn, nltk, dan sklearn yang umum digunakan pada data processing</p>

In [1]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


#stop-words
stop_words=set(nltk.corpus.stopwords.words('indonesian'))

<h3>Instalasi Library Tambahan</h3>
<p>Dibawah ini ada library tambahan yang harus diinstall untuk memroses kata-kata</p>

In [ ]:
nltk.download('corpus')
nltk.download('stopwords')
nltk.download('punkt')

<h3>Import dokumen</h3>

<p>Import dokumen yang sudah dicrawling dengan crawler, bisa menggunakan referensi dari web ini atau bisa menggunakan referensi kode dari website lain</p>

In [2]:
df=pd.read_csv(r'abstrak.csv')

<h3>Display 10 data teratas pada dokumen</h3>

In [3]:
df.head(10)

,judul,Abstrak,label
0,SISTEM PENENTUAN STATUS GIZI PASIEN RAWAT INAP...,Di Indonesia masalah perkembangan gizi adalah ...,CAI
1,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,RPL
2,RANCANG BANGUN APLIKASI INTRUSION PREVENTION S...,Dalam setiap perusahaan tentunya memiliki data...,RPL
3,PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM...,Penyusunan Sistem Informasi Dinas Perindustria...,CAI
4,APLIKASI DAN DESAIN MODEL DECISION AID PELANGG...,Toko Batik Tulis Madura merupakan salah satu t...,RPL
5,Gerak Pekerja Pada Game Real Time Strategy Men...,Gerak pekerja ada pada game yang memiliki genr...,CAI
6,RANCANG BANGUN SISTEM INFORMASI PEMETAAN TANAM...,Kabupaten Bangkalan memiliki lahan pertanian s...,RPL
7,PERANCANGAN SISTEM INFORMASI PERENCANAAN PRODU...,Penelitian ini membahas tentang perancangan si...,RPL
8,RANCANG BANGUN APLIKASI SEARCH ENGINE DAN SIST...,"Plagiarisme adalah mencuri gagasan, kata-kata,...",RPL
9,IMPLEMENTASI METODE N-GRAM PADA PEMBUATAN GAM...,Teknologi mobile game berkembang dengan sa...,CAI


In [4]:
def clean_text(headline):
    le=WordNetLemmatizer()
    word_tokens=word_tokenize(headline)
    tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
    cleaned_text=" ".join(tokens)
    return cleaned_text

<h3>Pembersihan dokumen</h3>

<p>pembersihan dokumen diperlukan agar supaya dalam proses TF/IDF tidak ada simbol-simbol yang masuk dalam proses tersebut yang mengakibatkan dokumen menjadi kurang otentik</p>

In [5]:
# time taking
df['abstrak_cleaned']=df['Abstrak'].apply(clean_text)

<h3>Perbandingan data yang belum dan sudah dibershkan</3>

In [6]:
df.head()

,judul,Abstrak,label,abstrak_cleaned
0,SISTEM PENENTUAN STATUS GIZI PASIEN RAWAT INAP...,Di Indonesia masalah perkembangan gizi adalah ...,CAI,Indonesia perkembangan gizi perhatian Jika sta...
1,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,RPL,Sistem informasi akademik SIAKAD sistem inform...
2,RANCANG BANGUN APLIKASI INTRUSION PREVENTION S...,Dalam setiap perusahaan tentunya memiliki data...,RPL,Dalam perusahaan memiliki data berkaitan kegia...
3,PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM...,Penyusunan Sistem Informasi Dinas Perindustria...,CAI,Penyusunan Sistem Informasi Dinas Perindustria...
4,APLIKASI DAN DESAIN MODEL DECISION AID PELANGG...,Toko Batik Tulis Madura merupakan salah satu t...,RPL,Toko Batik Tulis Madura salah toko beroperasi ...


<p>Dorp kolom Abstrak</p>

In [7]:
df.drop(['Abstrak'],axis=1,inplace=True)

In [8]:
df.head()

,judul,label,abstrak_cleaned
0,SISTEM PENENTUAN STATUS GIZI PASIEN RAWAT INAP...,CAI,Indonesia perkembangan gizi perhatian Jika sta...
1,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,RPL,Sistem informasi akademik SIAKAD sistem inform...
2,RANCANG BANGUN APLIKASI INTRUSION PREVENTION S...,RPL,Dalam perusahaan memiliki data berkaitan kegia...
3,PEMANFAATAN TOGAF ADM UNTUK PERANCANGAN SISTEM...,CAI,Penyusunan Sistem Informasi Dinas Perindustria...
4,APLIKASI DAN DESAIN MODEL DECISION AID PELANGG...,RPL,Toko Batik Tulis Madura salah toko beroperasi ...


<p>Data frame kolom abstrak_cleaned</p>

In [9]:
df['abstrak_cleaned'][0]

'Indonesia perkembangan gizi perhatian Jika status gizinya mengontrol gizi dibutuhkan tubuh Dalam penelitian dirancang aplikasi sistem pendukung keputusan menentukan status gizi pasien solusi makanan pasien sesuai riwayat penyakit derita pasien Sistem dirancang berbasis memudahkan admin ahli gizi rumah sakit penentuan status gizi pasien Diharapkan aplikasi efisien efektifitas kinerja Metode penelitian Naïve Bayes Classifier Metode terbaru memprediksi probabilitas.Metode Naïve bayes Classifier prose penentuan perhitungan probabilitas status gizi Dimana dicari nilai probabilitas terbesar kesimpulan penentuan status gizi Metode diterapkan studi Sistem Penentuan Status Gizi Pasien hasil akurasi terbesar Kata Kunci Naïve Bayes Classifier Sistem Pendukung Keputusan Status Gizi'

In [10]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...

In [11]:
vect_text=vect.fit_transform(df['abstrak_cleaned'])

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [12]:
print(vect_text.shape)
vect_text = vect_text.transpose()
df = pd.DataFrame(vect_text.toarray())
print(df)

(53, 1000)
      0    1    2    3        4    5    6    7    8    9   ...   43        44  \
0    0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
1    0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
2    0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.065267   
3    0.0  0.0  0.0  0.0  0.06755  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
4    0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
..   ...  ...  ...  ...      ...  ...  ...  ...  ...  ...  ...  ...       ...   
995  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
996  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
997  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
998  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   
999  0.0  0.0  0.0  0.0  0.00000  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.000000   

           45   

<h2>Term Frequency</h2>

<p>Data di atas adalah data yang sudah diproses menggunakan TF-IDF untuk menentukan Term Frequency tiap topik. TF-IDF (Term Frequency - Inverse Document Frequency) adalah algoritma praktis yang menggunakan frekuensi kata untuk menentukan seberapa relevan kata-kata itu dengan dokumen tertentu. Ini adalah pendekatan yang relatif sederhana namun intuitif untuk pembobotan kata, memungkinkannya bertindak sebagai titik awal yang bagus untuk berbagai tugas.</p>
<h3>Rumus TF-IDF</h3>

$$
\operatorname{tf}(t, d)=\frac{f_{t, d}}{\sum_{t^{\prime} \in d} f_{t^{\prime}, d}}
$$

<p></p>

In [13]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['gizi'])
print(dd['indonesia'])  # police is most common and forecast is least common among the news headlines.

metode yale
4.295836866004329
2.9095425048844383


C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<h2>Proses LSA</h2>

<p>LSA  adalah metode yang memungkinkan kita mengekstrak topik dari dokumen dengan mengubah teksnya menjadi matriks topik-kata dan topik-dokumen. Prosedur untuk LSA relatif mudah: Ubah korpus teks menjadi matriks istilah dokumen. Menerapkan dekomposisi nilai singular terpotong.</p>

$A_{m n}=U_{m m} x S_{m n} x V_{n n}^{T}$

<p>Matriks U = baris merepresentasikan vektor pada topic dokumen</p>
<p>Matriks V = Garis ini merepresentasikan vektor istilah yang dinyatakan pada topik</p>
<p>Matriks S = Matriks diagonal yang memiliki elemen-elemen diagonal yang digunakan sebagai nilai singular A</p>

<p>tiap baris pada matriks U merupakan representasi vektor yang terdapat pada dokumen yang sesuai, untuk melakukannya dapat menggunakan library SKLearn yang bernama TruncatedSVD untuk menimplementasikan LSA</p>

In [28]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)
print(pd.DataFrame(lsa_top))
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

            0         1         2         3         4         5         6  \
0    0.022513  0.014103  0.001021 -0.032999 -0.008429  0.010007 -0.004351   
1    0.066848  0.036675 -0.006424  0.120559  0.006876  0.011078  0.014678   
2    0.050591  0.023327  0.003047  0.041339 -0.009654  0.040480 -0.007151   
3    0.015598 -0.020772  0.011073 -0.004783  0.007014 -0.003704 -0.014951   
4    0.028148 -0.034250  0.036041  0.010536 -0.029183 -0.007120 -0.039525   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.025739 -0.050619 -0.062993 -0.008465  0.020182  0.014993 -0.038840   
996  0.017284  0.019783 -0.003151  0.015708  0.006698 -0.000563 -0.005028   
997  0.019228 -0.016835  0.010406  0.005682 -0.066119 -0.066268 -0.012515   
998  0.003348 -0.004964  0.003263 -0.002041 -0.003350  0.011977  0.007883   
999  0.044566  0.024450 -0.004283  0.080373  0.004584  0.007386  0.009785   

            7         8         9  
0    0.002334  0.019817  0.003725  
1  

In [15]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  2.2512533847059077
Topic  1  :  1.4103483378595856
Topic  2  :  0.1021361522709356
Topic  3  :  -3.29989132987768
Topic  4  :  -0.8428659594988172
Topic  5  :  1.0007158831415892
Topic  6  :  -0.43505022159714185
Topic  7  :  0.23342382167115988
Topic  8  :  1.9816560404465673
Topic  9  :  0.37254435688536247


In [18]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(pd.DataFrame(lsa_model.components_))

(10, 53)
         0         1         2         3         4         5         6   \
0  0.068430  0.087569  0.048283  0.050515  0.069584  0.041484  0.070037   
1 -0.034432 -0.076677 -0.035018 -0.047440 -0.041196 -0.028298 -0.076400   
2  0.090693  0.089060  0.055761  0.039443  0.069585  0.049706  0.016740   
3  0.009532 -0.003035  0.009132 -0.026681 -0.034690 -0.009576  0.002577   
4 -0.008122  0.030229 -0.035918 -0.032988 -0.042260 -0.153400  0.013233   
5  0.006432 -0.047401 -0.031839  0.057835 -0.023972  0.204769  0.079421   
6 -0.067881  0.218470  0.141479  0.072707  0.122931 -0.108840  0.100418   
7 -0.019660 -0.026724 -0.019054 -0.090082 -0.017617  0.043999 -0.161656   
8 -0.160007  0.068005  0.042184  0.157873 -0.054934 -0.061805  0.099774   
9 -0.067242 -0.037838  0.031479 -0.158425 -0.153505  0.276874 -0.232607   

         7         8         9   ...        43        44        45        46  \
0  0.095057  0.109287  0.058001  ...  0.134977  0.086334  0.294137  0.108259   
1 -0.

<h3>Hasil TruncatedSVD</h3>

berikut adalah contoh 10 kata penting ditiap topik yang diproses

In [19]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
anak akurasi 89 angka 93 95 aliran 83 55 aksesoris 

Topic 1: 
anak 95 aliran angka akurasi 83 alphabet 89 93 akademik 

Topic 2: 
aplikasi alert alternatif 79 angin 64 ak algoritma 82 akurat 

Topic 3: 
93 89 akurasi anak 95 akurat 94 acuan 30 ak 

Topic 4: 
aplikasi alert alternatif alphanumerik 85 anak 71 95 akurasi alat 

Topic 5: 
84 49 algoritma ambang admin 23 adaptive analisis 64 85 

Topic 6: 
82 64 aksesoris 10 analysis 94 aktor 55 15 2013 

Topic 7: 
angin akurat ak aksesoris akademik 94 admin 88 analysis 83 

Topic 8: 
akademik 88 ambang 83 89 alert admin 2011 aplikasi 2dpca 

Topic 9: 
analysis 49 23 aksesoris 84 alert alternatif 47 acuan aplikasi 



C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<p>List di atas adalah list kata tiap topik yang sering keluar disetiap dokumen</p>

In [20]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
# n_components is the number of topics

In [21]:
lda_top=lda_model.fit_transform(vect_text)

In [22]:
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(pd.DataFrame(lda_top))

(1000, 10)
            0         1         2         3         4         5         6  \
0    0.087092  0.087167  0.134061  0.087059  0.087052  0.087050  0.169293   
1    0.077326  0.077294  0.077294  0.077294  0.077293  0.077296  0.077300   
2    0.076829  0.075987  0.314588  0.075982  0.075976  0.076000  0.076011   
3    0.085576  0.138646  0.085527  0.085524  0.085527  0.085506  0.085554   
4    0.076603  0.076601  0.076600  0.076601  0.076608  0.310569  0.076601   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.085299  0.085295  0.085299  0.085290  0.085288  0.085289  0.085286   
996  0.093216  0.092859  0.092838  0.092836  0.092871  0.092857  0.092877   
997  0.082317  0.082324  0.082314  0.082313  0.082319  0.082310  0.259161   
998  0.093930  0.094265  0.093983  0.153448  0.094120  0.093969  0.093936   
999  0.083687  0.083628  0.083629  0.083627  0.083625  0.083633  0.083640   

            7         8         9  
0    0.087128  0.087064  0.0

In [23]:
sum=0
for i in lda_top[0]:
  sum=sum+i
print(sum)

1.0000000000000002


In [24]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  8.709153481708022 %
Topic  1 :  8.716695034312814 %
Topic  2 :  13.406128303393967 %
Topic  3 :  8.705874168247345 %
Topic  4 :  8.70523288192736 %
Topic  5 :  8.704966483060026 %
Topic  6 :  16.929272220568716 %
Topic  7 :  8.71277056908348 %
Topic  8 :  8.70644228424931 %
Topic  9 :  8.703464573448978 %


In [25]:
print(pd.DataFrame(lda_model.components_))
print(lda_model.components_.shape)  # (no_of_topics*no_of_words)

         0         1         2         3         4         5         6   \
0  2.400921  0.438430  0.402438  0.396266  0.750646  0.413491  0.349262   
1  0.380390  0.333192  0.400007  0.354619  3.116947  0.338409  0.391648   
2  0.413179  3.504743  4.265508  0.419454  0.513505  0.714777  0.444852   
3  0.380846  0.712788  0.341327  3.133547  0.422094  0.352413  3.317880   
4  0.685820  0.607540  0.502858  0.429167  0.456768  0.421257  0.581143   
5  0.414249  0.317296  0.316882  0.483242  0.309278  2.286915  0.442019   
6  0.349023  0.369598  0.354725  0.330858  0.563234  0.346109  0.507389   
7  0.342231  0.338335  0.372045  0.392474  0.294119  0.267297  0.526056   
8  0.520493  0.306261  0.477029  0.420385  0.403138  0.551377  0.386706   
9  0.592538  0.496734  0.411082  0.419531  0.552470  0.403008  0.370583   

         7         8         9   ...        43        44        45        46  \
0  2.186223  0.671951  0.427236  ...  0.471393  0.485204  3.050398  0.731387   
1  0.295904  0

In [26]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
anak analysis akurasi 00 admin aksesoris 30 93 89 95 

Topic 1: 
2013 akademik alternatif 55 83 analisis 89 android 85 aplikasi 

Topic 2: 
15 10 analisa 94 adaptive 82 alat 84 53 algoritma 

Topic 3: 
2dpca 2011 96 alat 98 acuan analisis aksesoris 49 10 

Topic 4: 
79 akurat aktor 47 88 53 algoritma 30 ak 73 

Topic 5: 
73 ak 23 admin ambang 30 angka 2011 aksesoris 53 

Topic 6: 
android 83 64 89 30 aliran 82 47 93 2013 

Topic 7: 
alphabet alphanumerik angka 85 55 aliran 95 alat 83 84 

Topic 8: 
ambang 71 49 84 algoritma admin analisis ak analisa android 

Topic 9: 
angin aktivitas alert 93 aplikasi alternatif akurat ak 64 algoritma 



C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
